# P3 - Movie Genre Classification

Elaborado por GRUPO 1:

- Juanita Piraban Barbosa - 201216313
- Lorena Morales Rodríguez - 202027957
- Alejandro Barinas Guio - 201628859
- Jaime Humberto Trujillo Perea - 201920366
- Alexander Zapata Galindo - 201425426

### Objective

Classify a movie genre based on its plot.
Source: https://www.kaggle.com/c/miia4201-202019-p3-moviegenreclassification/overview

Input: movie plot
Output: Probability of the movie belong to each genre

### Evaluation

- 20% API
- 30% Report with all the details of the solution, the analysis and the conclusions. The report cannot exceed 10 pages, must be send in PDF format and must be self-contained.
- 50% Performance in the Kaggle competition (The grade for each group will be proportional to the ranking it occupies in the competition. The group in the first place will obtain 5 points, for each position below, 0.25 points will be subtracted, that is: first place: 5 points, second: 4.75 points, third place: 4.50 points ... eleventh place: 2.50 points, twelfth place: 2.25 points).

• The project must be carried out in the groups assigned for module 4.
• Use clear and rigorous procedures.
• The delivery of the project is on July 12, 2020, 11:59 pm, through Sicua + (Upload: the API and the report in PDF format).
• No projects will be received after the delivery time or by any other means than the one established. 

### Acknowledgements

We thank Professor Fabio Gonzalez, Ph.D. and his student John Arevalo for providing this dataset.
See https://arxiv.org/abs/1702.01992

## Sample Submission

In [ ]:
# Librerías

import pandas as pd
from matplotlib import pyplot as plt
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

import keras
from keras import backend as K

import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from livelossplot import PlotLossesKeras
%matplotlib inline


In [ ]:
#pip install tensorflow

In [ ]:
#pip install livelossplot

In [ ]:
df_train = pd.read_csv('DataSet/dataTraining.csv', encoding='UTF-8', index_col=0)
df_test = pd.read_csv('Dataset/dataTesting.csv', encoding='UTF-8', index_col=0)

In [ ]:
df_train.head()

In [ ]:
df_train['rating'] = df_train['rating'].astype(int)
df_train.info()

In [ ]:
df_test.head()

In [ ]:
df_test.info()

### Limpieza y Normalización

In [ ]:
## eliminar caracteres especiales
#spec_chars = ["!",'"',"#","%","&","'","(",")",
#              "*","+",",","-",".","/",":",";","<",
#              "=",">","?","@","[","\\","]","^","_",
#              "`","{","|","}","~","–"]
#cols = df_train.select_dtypes(include=[np.object]).columns
#for char in spec_chars:
#    df_train[cols] = df_train[cols].apply(lambda x: x.str.replace(char, ''))
#    
#cols = df_test.select_dtypes(include=[np.object]).columns
#for char in spec_chars:
#    df_test[cols] = df_test[cols].apply(lambda x: x.str.replace(char, ''))

In [ ]:
# eliminar minúsculas
cols = df_train.select_dtypes(include=[np.object]).columns
df_train[cols] = df_train[cols].apply(lambda x: x.str.lower())

cols = df_test.select_dtypes(include=[np.object]).columns
df_test[cols] = df_test[cols].apply(lambda x: x.str.lower())

In [ ]:
# eliminar tildes
cols = df_train.select_dtypes(include=[np.object]).columns
df_train[cols] = df_train[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

cols = df_test.select_dtypes(include=[np.object]).columns
df_test[cols] = df_test[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

In [ ]:
df_train.head()

### Descriptive Analysis

In [ ]:
print('year:')
df_train['year'].value_counts()

In [ ]:
histogram = df_train['year'].plot.hist()
print(histogram)
plt.show()

In [ ]:
print('rating:')
df_train['rating'].value_counts()

In [ ]:
histogram = df_train['rating'].plot.hist()
print(histogram)
plt.show()

### Create count vectorizer


In [ ]:
vect = CountVectorizer(max_features=1000)
X_dtm = vect.fit_transform(df_train['plot'])
X_dtm.shape

In [ ]:
print(vect.get_feature_names())

### Create y

In [ ]:
df_train['genres'] = df_train['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(df_train['genres'])


In [ ]:
y_genres

In [ ]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train_genres.shape

In [ ]:
y_test_genres

### Train multi-class multi-label model

In [ ]:
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))

In [ ]:
clf.fit(X_train, y_train_genres)

In [ ]:
y_pred_genres = clf.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

### Predict the testing dataset

In [ ]:
X_test_dtm = vect.transform(df_test['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = clf.predict_proba(X_test_dtm)


In [ ]:
res = pd.DataFrame(y_pred_test_genres, index=df_test.index, columns=cols)

In [ ]:
res.head()

In [ ]:
res.to_csv('pred_genres_text_RF.csv', index_label='ID')

### tunning 

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=100, bootstrap=True, random_state=1234)


In [ ]:
# hiperparameters
max_features = ['auto', 'sqrt'] # Number of features to consider at every split
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] # Maximum number of levels in tree
max_depth.append(None)
min_samples_split = [2, 5, 10] # Minimum number of samples required to split a node
min_samples_leaf = [1, 2, 3, 4] # Minimum number of samples required at each leaf node
parameters = {
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf
            }
# grid
model_tunning = GridSearchCV(estimator=rfc, param_grid=parameters, scoring=make_scorer(roc_auc_score), cv=10)
clf = OneVsRestClassifier(estimator=model_tunning)
# model
clf.fit(X_train, y_train_genres)

In [ ]:
model_tunning.best_score_
model_tunning.best_params_

# Red Neuronal

In [ ]:
# Crear el vocabulario
X= df_train['plot'].tolist()

In [ ]:
X_train

In [ ]:
# For vocabulary only the intersec characters is used to avoid issues with data collection
voc = set(''.join(X))
vocabulary = {x: idx + 1 for idx, x in enumerate(set(voc))}

In [ ]:
vocabulary

In [ ]:
# Max len
max_plot_len = 10000
X = [x[:max_plot_len] for x in X]
# Convert characters to int and pad
X = [[vocabulary[x1] for x1 in x if x1 in vocabulary.keys()] for x in X]

In [ ]:
X[5]

In [ ]:
X_pad = sequence.pad_sequences(X, maxlen=max_plot_len)

In [ ]:
X_pad

In [ ]:
#Crear la red
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_pad, y_genres, test_size=0.33, random_state=42)

In [ ]:
from keras.layers import Activation, Dense, Flatten 

In [ ]:
model = Sequential()
model.add(Embedding(len(vocabulary) + 1, 128, input_length=max_plot_len))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1, input_dim=24, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

In [ ]:
i#mport scipy.sort_indices
#scipy.sort_indices(X_train)

In [ ]:
model.fit(X_train, y_train_genres, validation_data=[X_test, y_test_genres], 
          batch_size=128, epochs=10, verbose=1,
          callbacks=[PlotLossesKeras()])